In [20]:
from USTs import USTs
import datetime
from datetime import timedelta
import QuantLib as ql
import numpy as np
from typing import List
import calendar
from dateutil.relativedelta import relativedelta

In [3]:
# Bill pricing
issue_date = datetime.date(2004, 1, 22)
maturity_date = datetime.date(2004, 2, 19)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.937778,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1

0.8

In [4]:
issue_date = datetime.date(2004, 1, 22)
maturity_date = datetime.date(2004, 2, 19)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.937778,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

0.811

In [5]:
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [ ]:
issue_date = datetime.date(2015, 8, 17)
maturity_date = datetime.date(2025, 8, 15)

def adjust_for_weekend(date: datetime.date) -> datetime.date: # type: ignore
    if date.weekday() in [5, 6]:
        date = date + timedelta(days=(7 - date.weekday()))
    return date

def get_coupon_dates(issue_date, maturity_date) -> List[datetime.date]: # type: ignore
    payment_set = []
    payment_set.append(maturity_date)
    current_date = maturity_date

    while current_date > issue_date:
        current_date = maturity_date - relativedelta(months=len(payment_set) * 6)
        current_date = adjust_for_weekend(current_date)
        if current_date > issue_date:
            payment_set.append(current_date)
    payment_set.sort()
    return payment_set

In [ ]:
efg = get_coupon_dates(issue_date, maturity_date)
efg

[datetime.date(2016, 2, 15),
 datetime.date(2016, 8, 15),
 datetime.date(2017, 2, 15),
 datetime.date(2017, 8, 15),
 datetime.date(2018, 2, 15),
 datetime.date(2018, 8, 15),
 datetime.date(2019, 2, 15),
 datetime.date(2019, 8, 15),
 datetime.date(2020, 2, 17),
 datetime.date(2020, 8, 17),
 datetime.date(2021, 2, 15),
 datetime.date(2021, 8, 16),
 datetime.date(2022, 2, 15),
 datetime.date(2022, 8, 15),
 datetime.date(2023, 2, 15),
 datetime.date(2023, 8, 15),
 datetime.date(2024, 2, 15),
 datetime.date(2024, 8, 15),
 datetime.date(2025, 2, 17),
 datetime.date(2025, 8, 15)]

In [63]:
datetime.date(2025, 5, 31) - relativedelta(months=7)

datetime.date(2024, 10, 31)

0